In [1]:
import numpy as np
import tensorflow as tf
from string import punctuation
from collections import Counter
import collections
import random
import math
from six.moves import xrange 
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
import csv
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.layers import fully_connected

In [19]:
seq_len=200
embeding_size=128
n_steps = seq_len
n_inputs = embeding_size
n_neurons = 150
n_outputs = 2
learning_rate = 0.001
nbr_layers=3

In [15]:
"""
i=0
for chunk in pd.read_csv('Seq_Embo.csv',header=None,chunksize=200):
    chunk.to_csv('prodiff/Seq_'+str(i)+'.csv',index=None,header=None)
    if i%5==0:
        print(i)
    i=i+1
j=0
for chunk in pd.read_csv('labels.csv',header=None,chunksize=200):
    chunk.to_csv('prodiff/lab_'+str(j)+'.csv',index=None,header=None)
    if j%5:
        print(j)
    j=j+1
"""

"\ni=0\nfor chunk in pd.read_csv('Seq_Embo.csv',header=None,chunksize=200):\n    chunk.to_csv('prodiff/Seq_'+str(i)+'.csv',index=None,header=None)\n    if i%5==0:\n        print(i)\n    i=i+1\nj=0\nfor chunk in pd.read_csv('labels.csv',header=None,chunksize=200):\n    chunk.to_csv('prodiff/lab_'+str(j)+'.csv',index=None,header=None)\n    if j%5:\n        print(j)\n    j=j+1\n"

In [4]:
with open('lab.txt', 'r') as f:
    labels_org = f.read()
    

from sklearn.model_selection import train_test_split

train_x,XX,train_y,YY=train_test_split(features,labels,test_size=0.2)
val_x, test_x,val_y, test_y=train_test_split(XX,YY,test_size=0.5)



In [5]:
X_test.shape

(200, 200, 128)

In [20]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])
keep_drop=tf.placeholder(tf.float32, name="keep_prob")
def Cell():
    basic_cell = tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons)
    drop_cell=tf.contrib.rnn.DropoutWrapper(basic_cell, output_keep_prob=keep_drop)
    return basic_cell
temp=[Cell() for _ in range(nbr_layers)]
cells=tf.contrib.rnn.MultiRNNCell(temp)
outputs, states = tf.nn.dynamic_rnn(cells, X, dtype=tf.float32)

for one_lstm_cell in temp:
    one_kernel, one_bias = one_lstm_cell.variables
    # I think TensorBoard handles summaries with the same name fine.
    tf.summary.histogram("Kernel", one_kernel)
    tf.summary.histogram("Bias", one_bias)

logits = fully_connected(outputs[:,-1],n_outputs, activation_fn=None)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

tf.summary.scalar('accuracy', accuracy)
tf.summary.scalar('loss', loss)
merged = tf.summary.merge_all()
init = tf.global_variables_initializer()


In [21]:
n_epochs = 10
batch_index=(25000// 200)-1
saver = tf.train.Saver()
with tf.Session() as sess:
    init.run()
    print('Initialized')
    train_writer = tf.summary.FileWriter('train_III', sess.graph)
    for epoch in range(n_epochs):
        act_1=0
        act_2=0
        for io in range(batch_index):
            X_batch  = np.array(pd.read_csv('prodiff/Seq_'+str(io)+'.csv',header=None),dtype=np.float32).reshape((-1, n_steps, n_inputs))
            y_batch  = np.array(pd.read_csv('prodiff/lab_'+str(io)+'.csv',header=None),dtype=np.int32).reshape(-1)
            _,acc_train=sess.run([training_op,accuracy], feed_dict={X: X_batch, y: y_batch, keep_drop: 0.75})
            act_1+=acc_train
            act_2+=acc_train
            if io%25==0:
                print('Made-->',io,act_2/25)
                summary=merged.eval(feed_dict={X: X_batch, y: y_batch, keep_drop: 1})
                train_writer.add_summary(summary, io)
                act_2=0
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test, keep_drop: 1})
        print(epoch, "Train accuracy:", act_1/batch_index, "Test accuracy:", acc_test)
        saver.save(sess, "checkpoints/sento-anal.ckpt")

"""
    from matplotlib import pyplot as plt
    o=outputs.eval(feed_dict={X: X_batch, y: y_batch})
    l=states.eval(feed_dict={X: X_batch, y: y_batch})
    for i in range(10):
        plt.imshow(o[i,:,:])
        plt.show()
"""

Initialized
Made--> 0 0.0201999998093
Made--> 25 0.512600001097
Made--> 50 0.545199999809
Made--> 75 0.554399996996
Made--> 100 0.545000004768
0 Train accuracy: 0.541370967703 Test accuracy: 0.535
Made--> 0 0.0211999988556
Made--> 25 0.575000001192
Made--> 50 0.561400001049
Made--> 75 0.561999995708
Made--> 100 0.565999996662
1 Train accuracy: 0.566653223528 Test accuracy: 0.54
Made--> 0 0.0218000006676
Made--> 25 0.583399999142
Made--> 50 0.570000004768


KeyboardInterrupt: 